In [1]:
import sqlite3

In [2]:
def remove_outliers(db_path, threshold=10000):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Identify trials with objective values greater than the threshold
    cursor.execute("SELECT trial_id, value FROM trial_values WHERE value > ?", (threshold,))
    outliers = cursor.fetchall()

    if not outliers:
        print("No outliers found.")
        conn.close()
        return

    print(f"Found {len(outliers)} outliers. Removing...")

    # Remove the identified outliers
    for trial_id, value in outliers:
        print(f"Removing trial_id: {trial_id}, value: {value}")
        cursor.execute("DELETE FROM trials WHERE trial_id = ?", (trial_id,))
        cursor.execute("DELETE FROM trial_params WHERE trial_id = ?", (trial_id,))
        cursor.execute("DELETE FROM trial_values WHERE trial_id = ?", (trial_id,))
        cursor.execute("DELETE FROM trial_user_attributes WHERE trial_id = ?", (trial_id,))
        cursor.execute("DELETE FROM trial_system_attributes WHERE trial_id = ?", (trial_id,))
        cursor.execute("DELETE FROM trial_intermediate_values WHERE trial_id = ?", (trial_id,))
        cursor.execute("DELETE FROM trial_heartbeats WHERE trial_id = ?", (trial_id,))

    # Commit the changes and close the connection
    conn.commit()
    conn.close()

    print("Outliers removed successfully.")


In [3]:
def list_tables(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()

    print("Tables in the database:")
    for table in tables:
        print(table[0])

    conn.close()
    
    
def list_columns(db_path, table_name):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    cursor.execute(f"PRAGMA table_info({table_name});")
    columns = cursor.fetchall()

    print(f"Columns in the {table_name} table:")
    for column in columns:
        print(column)

    conn.close()

In [4]:
def delete_trials_by_number_or_state(db_path, trial_number=None, state=None):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    if trial_number is not None:
        # Identify trials with the specified number
        cursor.execute("SELECT trial_id FROM trials WHERE number = ?", (trial_number,))
        trials_to_delete = cursor.fetchall()
        print(f"Found {len(trials_to_delete)} trials with number {trial_number}. Removing...")

    elif state is not None:
        # Identify trials with the specified state
        cursor.execute("SELECT trial_id FROM trials WHERE state = ?", (state,))
        trials_to_delete = cursor.fetchall()
        print(f"Found {len(trials_to_delete)} trials with state {state}. Removing...")

    else:
        print("No criteria specified for deletion.")
        conn.close()
        return

    if not trials_to_delete:
        print("No matching trials found.")
        conn.close()
        return

    # Remove the identified trials
    for (trial_id,) in trials_to_delete:
        print(f"Removing trial_id: {trial_id}")
        cursor.execute("DELETE FROM trials WHERE trial_id = ?", (trial_id,))
        cursor.execute("DELETE FROM trial_params WHERE trial_id = ?", (trial_id,))
        cursor.execute("DELETE FROM trial_values WHERE trial_id = ?", (trial_id,))
        cursor.execute("DELETE FROM trial_user_attributes WHERE trial_id = ?", (trial_id,))
        cursor.execute("DELETE FROM trial_system_attributes WHERE trial_id = ?", (trial_id,))
        cursor.execute("DELETE FROM trial_intermediate_values WHERE trial_id = ?", (trial_id,))
        cursor.execute("DELETE FROM trial_heartbeats WHERE trial_id = ?", (trial_id,))

    # Commit the changes and close the connection
    conn.commit()
    conn.close()

    print("Specified trials removed successfully.")

In [5]:
# Path to your SQLite database
db_path = "/data3/lsf/Pein/Power-Prediction/optuna_results/24-08-01/24-08-01-farm_dataset.db"


In [6]:

# List columns in the relevant tables

# list_tables(db_path)
# list_columns(db_path, "trials")
# list_columns(db_path, "trial_values")


In [7]:

# trial_id = 0
# delete_trials_by_number_or_state(db_path, trial_number=0)

# Delete trials by state
# delete_trials_by_number_or_state(db_path, state='Fail')

In [8]:
remove_outliers(db_path,1000)

Found 1 outliers. Removing...
Removing trial_id: 283, value: 1114.164599609375
Outliers removed successfully.


In [9]:
import optuna
from optuna.importance import get_param_importances

def analyze_hyperparameter_importance(study):
    # Compute hyperparameter importance
    importance = get_param_importances(study)

    print("Hyperparameter importance:")
    for param, imp in importance.items():
        print(f"{param}: {imp}")

# Path to your SQLite database
db_path = "/data3/lsf/Pein/Power-Prediction/optuna_results/24-07-21/24-07-21-search.db"
study_name = "24-07-21-search"  # Study name

# Load the existing study
study = optuna.load_study(study_name=study_name, storage=f"sqlite:///{db_path}")

# Analyze hyperparameter importance
analyze_hyperparameter_importance(study)

/home/lsf/anaconda3/envs/Pein_310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Hyperparameter importance:
